# Doodler

This notebook shows how the `Doodler` application can be built on top of the tools provided by the Holoviz ecosystem. Small components are built successively to add functionality to the application. They are put together in a main class and finally organized in the desired layouts, one for testing directly in the notebook and one that is meant to be deployed.

In [ ]:
import configparser

import holoviews as hv
import panel as pn

# Import the components
from doodler.components import (
    Application,
    ClassToggleGroup,
    ComputationSettings,
    DoodleDrawer,
    Info,
    InputImage,
    Toggle,
)

## Configuration

The class/color mapping is obtained from an INI file.

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
CLASS_COLOR_MAPPING = dict(config['classes'])
CLASS_COLOR_MAPPING

## Components

### Class toggle group

The `ClassToggleGroup` component allows to toggle a class by clicking on its colorized button. The advantage of this component is that the buttons are layed out in a flexible container that wraps them line by line responsively.

We first define an `Toggle` component that represents a button of the `ClassToggleGroup` component.

In [ ]:
t = Toggle(klass='water', color='blue')
t

In [ ]:
t.active = True

In [ ]:
t.active = False

In [ ]:
ctg = ClassToggleGroup(class_color_mapping=CLASS_COLOR_MAPPING)
ctg

The `active` parameter stores the currently toggled class.

In [ ]:
ctg.active

### DoodleDrawer

The `DoodleDrawer` class provides the drawing functionality required for `Doodler`, i.e. the ability to quickly draw lines with different class/color and width. Its `doodles` property allows to obtain the lines drawn as a list of pandas dataframes.

In [ ]:
doodle_drawer = DoodleDrawer(class_color_mapping=CLASS_COLOR_MAPPING, class_toggle_group_type=ClassToggleGroup)

In [ ]:
pn.Row(
    pn.Column(
        doodle_drawer.class_toggle_group,
        doodle_drawer.param.line_width,
        doodle_drawer.param.clear_all,
    ),
    doodle_drawer.plot.opts(width=600),
)

In [ ]:
len(doodle_drawer.doodles)

In [ ]:
try:
    print(doodle_drawer.doodles[0].head())
except IndexError:
    pass

### Input image

The `InputImage` component allows a user to select an image. An instance can be created with the `from_folder` class method that will find all the JPEG images in a folder. The `remove_img` method removes the current image from the list of images available and sets the next one, if available.

In [ ]:
input_image = InputImage.from_folder('examples/images')

In [ ]:
pn.Row(input_image.param, input_image.pane)

In [ ]:
input_image.remove_img()

In [ ]:
input_image.remove_img()

In [ ]:
input_image.remove_img()

In [ ]:
input_image.remove_img()

### Computation settings

The `ComputationSettings` class declares all the parameters required by the algorithms perfoming the segmentation. UI-wise it provides the ability to switch to an *advanced* mode that displays more parameters to the user.

In [ ]:
cs = ComputationSettings()
cs

The `as_dict` method is useful to get the values of all the parameters.

In [ ]:
cs.as_dict()

### Info

The `Info` class renders as an *Alert* viewable component that is useful to show messages of different types to the user.

In [ ]:
i = Info()
i

In [ ]:
i.update('first line', 'warning')

In [ ]:
i.add('another line')

In [ ]:
i.reset()

## Combining the components with the segmentation computation

The `Application` class uses and combines the components introduced above with components and methods dedicated to the segmentation itself, that call the learning algorithms.

The different components are instantiated and passed to `Application`.

In [ ]:
doodle_drawer = DoodleDrawer(class_color_mapping=CLASS_COLOR_MAPPING, class_toggle_group_type=ClassToggleGroup)
input_image = InputImage.from_folder('examples/images')
settings = ComputationSettings(name='Post-processing/Classifier settings')
info = Info()
app = Application(settings=settings, doodle_drawer=doodle_drawer, info=info, input_image=input_image)

## Layout

### Notebook application

First a simple application is put together in the notebook by laying out the components in `Row` and `Column` Panel panes. This step is very useful when developing the application.

In [ ]:
side_bar = pn.Column(
    app.input_image.param.location,
    pn.pane.HTML('<b>Doodling options</b>'),
    app.doodle_drawer.class_toggle_group,
    app.doodle_drawer.param.line_width,
    app.doodle_drawer.param.clear_all,
    app.settings,
    pn.widgets.Button.from_param(app.param.compute_segmentation, button_type='primary'),
    pn.widgets.Button.from_param(app.param.clear_segmentation, button_type='warning'),
    pn.widgets.Button.from_param(app.param.save_segmentation, button_type='success'),
    app.info,
    width=350,  # Width set to avoid issues with the class FlexBox. Slightly less than the side_bar width of the Material template (370).
)
main = app.plot_pane
nb_app = pn.Row(side_bar, main)
nb_app

## Deployable application

While the notebook application already provides all the functionnality we require, its design should be improved a little to make it a proper web app. We're embedding it into one of the templates provided by Panel and add a few elements like the USGS logo. Serve the application by running `panel serve doodler.ipynb --show`.

In [ ]:
template = pn.template.MaterialTemplate(
    title='Doodler',
    logo='assets/1280px-USGS_logo.png',
    header_background='#000000',
    sidebar=[side_bar],
    main=[main],
)
template.servable();